In [1]:
import numpy as np
import pandas as pd
import random
import math

from sklearn import preprocessing  # normalization library

### Robot Data
For the Pioneer 3DX.  You can modify this data. It depends on your robot's mechanical design and the sensor characteristics.

In [2]:
LMr = 381e-3  # distance between wheels [m]
R = 195e-3  # Radio for each wheel  [m]

v_max = 1  # max linear velocity [m/s]
v_min = 0.3  # min linear velocity [m/s]

Lt_max = 1.2  # Maximum curvature until reaching the desired angle [m/s]
Lt_min = 0.3  # Minimun curvature until reaching the desired angle [m/s]

v_mean = (v_max+v_min)/2  # mean velocity based in robot range
print('The mean velocity of robot is : ', v_mean)

The mean velocity of robot is :  0.65


### Obstacle Detection Area 

This data depends on the selected sensor characteristics. In this case, we set a range between $[dmin_1\quad dmax_1]\quad m$ and a desired orientation angle between $[-\pi\quad\pi]\quad rad$. You can choose this parameters.

In [3]:
min_d1 = 0.3  # min detection value of ultrasonic sensors
max_d1 = 1  # max detection value of ultrasonic sensors

### Data generation for sensor:



In [5]:
# Normal Sensor Data Gen
Obs_Data = [round(random.uniform(min_d1, max_d1), 4) for _ in range(4500*3)]

print(' Normal raw data :', Obs_Data[:2])

# Small Sensor Data Gen
Obs_Data_Small = [round(random.uniform(min_d1, max_d1-0.6), 4)
                for _ in range(1500*3)]

print(' Small raw data :', Obs_Data_Small[:2])

 Normal raw data : [0.8245, 0.7304]
 Small raw data : [0.3853, 0.3781]


In [9]:
Obs_Data_Comp = np.hstack((Obs_Data, Obs_Data_Small)) #Joint normal and small data

Obs_Data_Comp = np.array(Obs_Data_Comp).reshape(6000, 3) # Reorganize in 6000 x 3 array

Obs_nData=pd.DataFrame(Obs_Data_Comp,columns=['LS','FS','RS']) #Create a dataframe

Obs_nData.head(2)

,LS,FS,RS
0,0.8245,0.7304,0.9893
1,0.4444,0.5403,0.3601


In [7]:
# Normal Theta Data
Theta_d = [round(random.uniform(-math.pi, math.pi), 4) for _ in range(4500)]

print(' Normal Theta data :', Theta_d[:2])

# Small Theta Data

Theta_d_Small = [round(random.uniform(-0.5, 0.5), 4) for _ in range(1500)]

print(' Small Theta data :', Theta_d[:2])

 Normal Theta data : [0.3456, -2.4985]
 Small Theta data : [0.3456, -2.4985]


In [8]:
Theta_data_Comp = np.hstack((Theta_d, Theta_d_Small)) # Joint normal angles and small angles


Theta_data_Comp = np.array(Theta_data_Comp).reshape(6000, 1) #Reshape in 6000 rows x one column

Thetadf=pd.DataFrame(Theta_data_Comp,columns=['angle']) # Create an pandas dataframe

Thetadf.head(3)

,angle
0,0.3456
1,-2.4985
2,1.3880


Create a complete dataframe 

In [10]:
Df_C1_Data = np.hstack((Obs_Data_Comp, Theta_data_Comp))

Df_C1 = pd.DataFrame(Df_C1_Data, columns=['LS', 'FS', 'RS', 'Theta_d'])
Df_C1.sample(6)

Df_C1.to_csv('Input_Data.csv', index=False)

Normalize all dataset using sklearn librarie 

In [11]:
scaler = preprocessing.MinMaxScaler()


normalized_C1 = pd.DataFrame(scaler.fit_transform(
    Df_C1), columns=['LS', 'FS', 'RS', 'Theta_d'])

normalized_C1 = normalized_C1.round(4)

normalized_C1.sample(6)

,LS,FS,RS,Theta_d
199,0.8687,0.3614,0.7574,0.8680
1046,0.8910,0.0466,0.8542,0.6847
5211,0.0091,0.0083,0.0462,0.4791
890,0.1982,0.8589,0.9053,0.5723
1456,0.9744,0.8674,0.0097,0.3674
5379,0.0673,0.1052,0.0546,0.4701


### Inverse Kinematic Model

$$\omega_{R/L}=\left( 1\pm\frac{b}{2L_T}\right)\frac{v}{R}$$

Function to give more gain if robot is near extremes (really near to obstacles and the opposite case)

In [12]:
def gain_near_extremes_normalized(x):
    return 1-(4 * x * (1 - x))  #inverse cuadratic function x^2

Function to give more gain if orientation angle is near extremes

In [13]:
def gain_near_zero(x, scale=10):
    gain = np.exp(-scale * x)
    normalized_gain = (gain - np.exp(-scale)) / (1 - np.exp(-scale))  # exponential function
    return normalized_gain

Denormalize values

In [19]:
def denormalize_single_value(normalized_value, feature_index, scaler):
    dummy_normalized_array = np.zeros((1, len(scaler.data_min_)))
    dummy_normalized_array[0, feature_index] = normalized_value
    denormalized_array = scaler.inverse_transform(dummy_normalized_array)
    denormalized_value = denormalized_array[0, feature_index]
    return denormalized_value

### Conditional Definitions 

In [16]:
# Case 1 L/R


def Case_1_R(odp, theta_norm):
    feature_index = normalized_C1.columns.get_loc('Theta_d')
    theta_d = denormalize_single_value(
        theta_norm, feature_index, scaler).round(4)
    if theta_d == 0:
        theta_d = 0.001
    S = -1
    factor = round(gain_near_extremes_normalized(odp), 4)
    Lt_p = round(((odp)*(Lt_max-Lt_min)+Lt_min), 4)
    V_p = round(v_max-(factor*(v_mean/2)), 4)
    if odp < 0.6 and theta_d < 0.5:
        fth = round(gain_near_zero(theta_d, scale=10), 4)
        theta_d = round(fth*math.pi, 4)
    else:
        theta_d = theta_d
    return round((1+((LMr*theta_d*S)/(2*Lt_p)))*(V_p/R), 4)


def Case_1_L(odp, theta_norm):
    feature_index = normalized_C1.columns.get_loc('Theta_d')
    theta_d = denormalize_single_value(
        theta_norm, feature_index, scaler).round(4)
    if theta_d == 0:
        theta_d = 0.001
    S = -1
    factor = round(gain_near_extremes_normalized(odp), 4)
    Lt_p = round(((odp)*(Lt_max-Lt_min)+Lt_min), 4)
    V_p = round(v_max-(factor*(v_mean/2)), 4)
    if odp < 0.6 and theta_d < 0.5:
        fth = round(gain_near_zero(theta_d, scale=10), 4)
        theta_d = round(fth*math.pi, 4)
    else:
        theta_d = theta_d
    return round((1-((LMr*theta_d*S)/(2*Lt_p)))*(V_p/R), 4)


# Case 2 L/R

def Case_2_R(odp, theta_norm):
    feature_index = normalized_C1.columns.get_loc('Theta_d')
    theta_d = denormalize_single_value(
        theta_norm, feature_index, scaler).round(4)
    if theta_d == 0:
        theta_d = -0.001
    S = 1
    factor = round(gain_near_extremes_normalized(odp), 4)
    Lt_p = round(((odp)*(Lt_max-Lt_min)+Lt_min), 4)
    V_p = round(v_max-(factor*(v_mean/2)), 4)
    if odp < 0.6 and theta_d > -0.5 and theta_d < 0:
        fth = round(gain_near_zero(-1*theta_d), 4)
        theta_d = fth*-math.pi
    else:
        theta_d = theta_d
    cf = math.pi/4
    return round((1+((LMr*(theta_d+cf)*S)/(2*Lt_p)))*(V_p/R), 4)


def Case_2_L(odp, theta_norm):
    feature_index = normalized_C1.columns.get_loc('Theta_d')
    theta_d = denormalize_single_value(
        theta_norm, feature_index, scaler).round(4)
    if theta_d == 0:
        theta_d = -0.001
    S = 1
    factor = round(gain_near_extremes_normalized(odp), 4)
    Lt_p = round(((odp)*(Lt_max-Lt_min)+Lt_min), 4)
    V_p = round(v_max-(factor*(v_mean/2)), 4)
    if odp < 0.6 and theta_d > -0.5 and theta_d < 0:
        fth = round(gain_near_zero(-1*theta_d), 4)
        theta_d = fth*-math.pi
    else:
        theta_d = theta_d
    cf = math.pi/4
    return round((1-((LMr*(theta_d+cf)*S)/(2*Lt_p)))*(V_p/R), 4)


# Case 3 L/R

def Case_3_R(odp, theta_norm):
    feature_index = normalized_C1.columns.get_loc('Theta_d')
    theta_d = denormalize_single_value(
        theta_norm, feature_index, scaler).round(4)
    if theta_d == 0:
        theta_d = 0.001
    S = 1
    factor = round(gain_near_extremes_normalized(odp), 4)
    Lt_p = round(((odp)*(Lt_max-Lt_min)+Lt_min), 4)
    V_p = round(v_max-(factor*(v_mean/2)), 4)
    if odp < 0.6 and theta_d < 0.5 and theta_d > 0:
        fth = round(gain_near_zero(theta_d), 4)
        theta_d = round(fth*math.pi)
    else:
        theta_d = theta_d
    cf = -math.pi/4
    return round((1+((LMr*(theta_d+cf)*S)/(2*Lt_p)))*(V_p/R), 4)


def Case_3_L(odp, theta_norm):
    feature_index = normalized_C1.columns.get_loc('Theta_d')
    theta_d = denormalize_single_value(
        theta_norm, feature_index, scaler).round(4)
    if theta_d == 0:
        theta_d = 0.001
    S = 1
    factor = round(gain_near_extremes_normalized(odp), 4)
    Lt_p = round(((odp)*(Lt_max-Lt_min)+Lt_min), 4)
    V_p = round(v_max-(factor*(v_mean/2)), 4)
    if odp < 0.6 and theta_d < 0.5 and theta_d > 0:
        fth = round(gain_near_zero(theta_d), 4)
        theta_d = round(fth*math.pi)
    else:
        theta_d = theta_d
    cf = -math.pi/4
    return round((1-((LMr*(theta_d+cf)*S)/(2*Lt_p)))*(V_p/R), 4)

# Case 4 L/R


def Case_4_R(odp, theta_norm):
    feature_index = normalized_C1.columns.get_loc('Theta_d')
    theta_d = denormalize_single_value(
        theta_norm, feature_index, scaler).round(4)
    if theta_d == 0:
        theta_d = -0.001
    S = -1
    factor = round(gain_near_extremes_normalized(odp), 4)
    Lt_p = round(((odp)*(Lt_max-Lt_min)+Lt_min), 4)
    V_p = round(v_max-(factor*(v_mean/2)), 4)
    if odp < 0.6 and theta_d > -0.5 and theta_d < 0:
        fth = round(gain_near_zero(-1*theta_d), 4)
        theta_d = round(fth*-math.pi, 4)
    else:
        theta_d = theta_d

    return round((1+((LMr*theta_d*S)/(2*Lt_p)))*(V_p/R), 4)


def Case_4_L(odp, theta_norm):
    feature_index = normalized_C1.columns.get_loc('Theta_d')
    theta_d = denormalize_single_value(
        theta_norm, feature_index, scaler).round(4)
    if theta_d == 0:
        theta_d = -0.001
    S = -1
    factor = round(gain_near_extremes_normalized(odp), 4)
    Lt_p = round(((odp)*(Lt_max-Lt_min)+Lt_min), 4)
    V_p = round(v_max-(factor*(v_mean/2)), 4)
    if odp < 0.6 and theta_d > -0.5 and theta_d < 0:
        fth = round(gain_near_zero(-1*theta_d), 4)
        theta_d = round(fth*-math.pi, 4)
    else:
        theta_d = theta_d

    return round((1-((LMr*theta_d*S)/(2*Lt_p)))*(V_p/R), 4)

# Case 5 L/R


def Case_5_R(odp, theta_norm):
    feature_index = normalized_C1.columns.get_loc('Theta_d')
    theta_d = denormalize_single_value(
        theta_norm, feature_index, scaler).round(4)
    if theta_d == 0:
        theta_d = 0.001
    S = -1
    factor = round(gain_near_extremes_normalized(odp), 4)
    Lt_p = round(((odp)*(Lt_max-Lt_min)+Lt_min), 4)
    V_p = round(v_max-(factor*(v_mean/2)), 4)
    if odp < 0.6 and theta_d < 0.5:
        fth = round(gain_near_zero(theta_d, scale=10), 4)
        theta_d = round(fth*math.pi, 4)
    else:
        theta_d = theta_d
    return round((1+((LMr*theta_d*S)/(2*Lt_p)))*(V_p/R), 4)


def Case_5_L(odp, theta_norm):
    feature_index = normalized_C1.columns.get_loc('Theta_d')
    theta_d = denormalize_single_value(
        theta_norm, feature_index, scaler).round(4)
    if theta_d == 0:
        theta_d = 0.001
    S = -1
    factor = round(gain_near_extremes_normalized(odp), 4)
    Lt_p = round(((odp)*(Lt_max-Lt_min)+Lt_min), 4)
    V_p = round(v_max-(factor*(v_mean/2)), 4)
    if odp < 0.6 and theta_d < 0.5:
        fth = round(gain_near_zero(theta_d, scale=10), 4)
        theta_d = round(fth*math.pi, 4)
    else:
        theta_d = theta_d
    return round((1-((LMr*theta_d*S)/(2*Lt_p)))*(V_p/R), 4)

# Case 6 L/R


def Case_6_R(odp, theta_norm):
    feature_index = normalized_C1.columns.get_loc('Theta_d')
    theta_d = denormalize_single_value(
        theta_norm, feature_index, scaler).round(4)
    if theta_d == 0:
        theta_d = -0.001
    S = -1
    factor = round(gain_near_extremes_normalized(odp), 4)
    Lt_p = round(((odp)*(Lt_max-Lt_min)+Lt_min), 4)
    V_p = round(v_max-(factor*(v_mean/2)), 4)
    if odp < 0.6 and theta_d > -0.5 and theta_d < 0:
        fth = round(gain_near_zero(-1*theta_d), 4)
        theta_d = round(fth*-math.pi, 4)
    else:
        theta_d = theta_d
    return round((1+((LMr*theta_d*S)/(2*Lt_p)))*(V_p/R), 4)


def Case_6_L(odp, theta_norm):
    feature_index = normalized_C1.columns.get_loc('Theta_d')
    theta_d = denormalize_single_value(
        theta_norm, feature_index, scaler).round(4)
    if theta_d == 0:
        theta_d = -0.001
    S = -1
    factor = round(gain_near_extremes_normalized(odp), 4)
    Lt_p = round(((odp)*(Lt_max-Lt_min)+Lt_min), 4)
    V_p = round(v_max-(factor*(v_mean/2)), 4)
    if odp < 0.6 and theta_d > -0.5 and theta_d < 0:
        fth = round(gain_near_zero(-1*theta_d), 4)
        theta_d = round(fth*-math.pi, 4)
    else:
        theta_d = theta_d
    return round((1-((LMr*theta_d*S)/(2*Lt_p)))*(V_p/R), 4)

Obtain min value

In [20]:
normalized_C1['Min Value'] = normalized_C1[['LS', 'FS', 'RS']].min(axis=1)
normalized_C1.sample(6)

,LS,FS,RS,Theta_d,Min Value
4149,0.1483,0.1305,0.6262,0.5929,0.1305
2972,0.5687,0.4691,0.1402,0.7417,0.1402
2053,0.4059,0.6170,0.3801,0.6437,0.3801
5533,0.0979,0.1329,0.0835,0.4607,0.0835
1725,0.6307,0.2190,0.3700,0.6713,0.2190
568,0.6194,0.1625,0.4974,0.0023,0.1625


###  Obtain Velocities

In [22]:
def lw_eval(df):
    if df['Min Value'] == df['LS'] and df['Theta_d'] >= 0.5:
        return Case_1_L(df['Min Value'], df['Theta_d'])
    elif df['Min Value'] == df['LS'] and df['Theta_d'] < 0.5:
        return Case_2_L(df['Min Value'], df['Theta_d'])
    elif df['Min Value'] == df['RS'] and df['Theta_d'] >= 0.5:
        return Case_3_L(df['Min Value'], df['Theta_d'])
    elif df['Min Value'] == df['RS'] and df['Theta_d'] < 0.5:
        return Case_4_L(df['Min Value'], df['Theta_d'])
    elif df['Min Value'] == df['FS'] and df['Theta_d'] >= 0.5:
        return Case_5_L(df['Min Value'], df['Theta_d'])
    elif df['Min Value'] == df['FS'] and df['Theta_d'] < 0.5:
        return Case_6_L(df['Min Value'], df['Theta_d'])



def rw_eval(df):
    if df['Min Value'] == df['LS'] and df['Theta_d'] >= 0.5:
        return Case_1_R(df['Min Value'], df['Theta_d'])
    elif df['Min Value'] == df['LS'] and df['Theta_d'] < 0.5:
        return Case_2_R(df['Min Value'], df['Theta_d'])
    elif df['Min Value'] == df['RS'] and df['Theta_d'] >= 0.5:
        return Case_3_R(df['Min Value'], df['Theta_d'])
    elif df['Min Value'] == df['RS'] and df['Theta_d'] < 0.5:
        return Case_4_R(df['Min Value'], df['Theta_d'])
    elif df['Min Value'] == df['FS'] and df['Theta_d'] >= 0.5:
        return Case_5_R(df['Min Value'], df['Theta_d'])
    elif df['Min Value'] == df['FS'] and df['Theta_d'] < 0.5:
        return Case_6_R(df['Min Value'], df['Theta_d'])

# applying functions to all rows taking the same row with different columns parameters 
normalized_C1 = normalized_C1.assign(
    vel_L=normalized_C1.apply(lw_eval, axis=1))
normalized_C1 = normalized_C1.assign(
    vel_R=normalized_C1.apply(rw_eval, axis=1))

normalized_C1_A=normalized_C1[['LS', 'FS', 'RS','Theta_d','vel_R','vel_L']]
normalized_C1_A.sample(6)


,LS,FS,RS,Theta_d,vel_R,vel_L
874,0.5622,0.0593,0.1303,0.7279,0.8774,6.7893
2766,0.7583,0.6517,0.7728,0.5891,4.3775,5.5723
3582,0.4632,0.8252,0.6905,0.5035,1.6619,8.5761
5285,0.1312,0.1261,0.0833,0.5416,2.3865,5.5550
5522,0.0366,0.0199,0.1095,0.5751,3.5309,3.6527
2765,0.7277,0.4948,0.5161,0.5308,4.5290,5.7274


Output Data Normalization 

In [23]:
normalized_C1_B=normalized_C1_A[['LS', 'FS', 'RS','Theta_d']]
output_data=normalized_C1_A[['vel_R', 'vel_L']]
output_data.head(6)

,vel_R,vel_L
0,4.6516,5.4284
1,9.0125,-1.0443
2,5.0629,2.5648
3,8.1127,1.7908
4,7.3186,1.9306
5,0.9010,7.8754


In [25]:
norm_output_data=scaler.fit_transform(output_data).round(4) # Normalize output data using min-max Scaler
df_n_Output=pd.DataFrame(norm_output_data,columns=['vel_R_norm', 'vel_L_norm'], index=normalized_C1_B.index)
normalized_C1_data=pd.concat([normalized_C1_B, df_n_Output], axis=1)
normalized_C1_data.head(6)

,LS,FS,RS,Theta_d,vel_R_norm,vel_L_norm
0,0.7496,0.6150,0.9849,0.5551,0.5879,0.6360
1,0.2064,0.3433,0.0857,0.1023,0.9054,0.1616
2,0.1941,0.0805,0.0560,0.7211,0.6178,0.4261
3,0.9514,0.7360,0.6626,0.0219,0.8399,0.3694
4,0.9814,0.7072,0.2251,0.8699,0.7821,0.3796
5,0.1668,0.5411,0.9273,0.0761,0.3147,0.8153


Train-Test split 

In [26]:
from sklearn.model_selection import train_test_split

In [28]:
train_C1, val_C1 = train_test_split(normalized_C1_data, test_size=0.2, random_state=42) 
#separate in 80% train and 20 % validation

Extract datasets for left wheel

In [29]:
lw_train=train_C1[['LS', 'FS', 'RS','Theta_d','vel_L_norm']]
lw_train.sample(6)

,LS,FS,RS,Theta_d,vel_L_norm
4779,0.1379,0.0636,0.0724,0.4992,0.0739
164,0.5874,0.2464,0.3294,0.7117,0.7496
25,0.4623,0.9560,0.1169,0.7024,0.4729
5601,0.0753,0.0146,0.0197,0.5562,0.5135
313,0.8888,0.5484,0.7898,0.9709,0.8787
5004,0.1075,0.1046,0.0059,0.4296,0.4887


In [30]:
lw_val=val_C1[['LS', 'FS', 'RS','Theta_d','vel_L_norm']]
lw_val.sample(6)

,LS,FS,RS,Theta_d,vel_L_norm
5364,0.0071,0.0672,0.0840,0.4978,0.8038
1684,0.8202,0.0775,0.2371,0.0600,0.1157
4930,0.1078,0.1083,0.0582,0.5694,0.6377
1973,0.5704,0.5682,0.4261,0.4339,0.6062
5948,0.0882,0.1228,0.0420,0.5132,0.4784
3970,0.9773,0.8774,0.3540,0.1685,0.3692


Extract datasets for right wheel


In [31]:
rw_train=train_C1[['LS', 'FS', 'RS','Theta_d','vel_R_norm']]
rw_train.sample(6)

,LS,FS,RS,Theta_d,vel_R_norm
4224,0.1565,0.5615,0.4364,0.7558,0.3459
3343,0.1096,0.8694,0.3542,0.0019,0.2132
5103,0.1082,0.0565,0.0094,0.5244,0.5397
708,0.4912,0.2631,0.1838,0.8525,0.7639
5967,0.0434,0.0960,0.1050,0.4965,0.2586
2607,0.3121,0.5105,0.9481,0.3823,0.6107


In [32]:
rw_val=val_C1[['LS', 'FS', 'RS','Theta_d','vel_R_norm']]
rw_val.sample(6)

,LS,FS,RS,Theta_d,vel_R_norm
3082,0.0292,0.0220,0.0247,0.4153,0.5950
1501,0.7506,0.8426,0.2128,0.8697,0.7810
5959,0.1005,0.0127,0.0014,0.5190,0.5362
4930,0.1078,0.1083,0.0582,0.5694,0.4095
3816,0.3887,0.7616,0.7428,0.2796,0.5520
2091,0.3596,0.5431,0.4444,0.9365,0.3083


To CSV files

In [33]:
# create directory to sava csv able to train in MATLAB
import os
parent_dir = 'Train_Val_Data_2'
child_dir_1 = os.path.join(parent_dir, 'Train')
child_dir_2 = os.path.join(parent_dir, 'Validation')
os.makedirs(child_dir_1, exist_ok=True)
os.makedirs(child_dir_2, exist_ok=True)

In [34]:
lw_train.to_csv(os.path.join(child_dir_1, 'C1_vl_train.csv'), index=False)
lw_val.to_csv(os.path.join(child_dir_2, 'C1_vl_val.csv'), index=False)


rw_train.to_csv(os.path.join(child_dir_1, 'C1_vr_train.csv'), index=False)
rw_val.to_csv(os.path.join(child_dir_2, 'C1_vr_val.csv'), index=False)